In [3]:
from sklearn.datasets import make_classification
from sklearn.cross_validation import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import numpy as np

/people/vu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# We use a utility to generate artificial classification data.
X, y = make_classification(n_samples=100, n_informative=10, n_classes=3)
sss = StratifiedShuffleSplit(y, n_iter=1, test_size=0.5, random_state=0)
for train_idx, test_idx in sss:
    X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))    

In [81]:
def calculate_AER(S, P, A):
    
    s_a = len(list(set(S).intersection(A)))
    print ("s_a", s_a)
    p_a = len(list(set(P).intersection(A))) + s_a
    print ("p_a", p_a)
    aer = (s_a + p_a) / (len(S) + len(A))
    print ("aer", 1-aer)
    
    return 1. - aer 
    
def write_file(strs, file_name):
    alignment_test = open(file_name,"w") 
    for s in english_sentences:
        alignment_test.write(s + "\n") 

    alignment_test.close()

In [35]:
S = ['3-2', '1-1']
P = ['2-2']
A = ['1-2', '1-1', '3-4']
calculate_AER(S, P, A)

s_a 1
p_a 1
aer 0.4


0.6

In [ ]:
<s id="celex_21990A0714-s1">
  <english>Relationship of ICOs and ICBs with the Fund</english>
  <czech>Vztah mezinárodních organizací pro suroviny a mezinárodních subjektů pro suroviny k fondu</czech>
  <sure>1-1 3-2 3-3 3-4 3-5 4-6 5-7 5-8 5-9 5-10 8-12</sure>
  <possible>2-3 6-11 7-12</possible>
  <phrasal></phrasal>
</s>

In [66]:
import xml.etree.ElementTree as ET
from xml.dom import minidom
from codecs import open

docXml = './celex_21990A0714.m.wa'
root = minidom.parse(docXml).getElementsByTagName("sentences")

english_sentences = []
czech_sentences = []
sure = []
possible = []

for sent in root[0].childNodes :
    
    #on retire les noeuds vides
    if sent.localName == None : continue

    sentID = sent.getAttribute('id')

    #pour chaque noeud de phrase on parcourt les tokens de la phrase, on regarde leur longueur et on calcule
    #le numero du premier charactère ... sera utile pour la suite pour avoir l'ID html du charactère correspondant
    tokenNodes = sent.childNodes
    
    english_sentences.append(tokenNodes[1].childNodes[0].data)
    czech_sentences.append(tokenNodes[3].childNodes[0].data)
    sure.append(tokenNodes[5].childNodes[0].data)
    try:
        possible.append(tokenNodes[7].childNodes[0].data)
    except Exception: 
        possible.append('')

        
write_file(english_sentences, "alignment_test.en")
write_file(czech_sentences, "alignment_test.cz")

In [87]:
result = open("corpus.en2cz.gdfa.ali", "r")
aer = 0.

for line, S_, P_ in zip(result, sure, possible):
    A = line.strip().split(" ")
    for number,i in zip(A, range(len(A))):
        n1 = int(number.split("-")[0]) + 1 
        n2 = int(number.split("-")[1]) + 1 
        number = str(n1) + "-" + str(n2)
        A[i] = number
    S = S_.strip().split(" ")
    P = P_.strip().split(" ")
    aer += calculate_AER(S, P, A)

print(aer/len(sure))
result.close()

s_a 2
p_a 2
aer 0.7894736842105263
s_a 8
p_a 14
aer 0.7215189873417722
s_a 2
p_a 4
aer 0.8571428571428572
s_a 3
p_a 3
aer 0.9411764705882353
s_a 4
p_a 7
aer 0.6764705882352942
s_a 0
p_a 1
aer 0.9818181818181818
s_a 2
p_a 2
aer 0.9069767441860466
s_a 2
p_a 6
aer 0.8431372549019608
s_a 1
p_a 3
aer 0.9540229885057472
s_a 0
p_a 3
aer 0.9285714285714286
s_a 1
p_a 1
aer 0.9696969696969697
s_a 0
p_a 1
aer 0.9444444444444444
s_a 0
p_a 0
aer 1.0
s_a 2
p_a 4
aer 0.9310344827586207
s_a 0
p_a 0
aer 1.0
s_a 1
p_a 1
aer 0.9130434782608696
s_a 2
p_a 3
aer 0.7916666666666666
s_a 8
p_a 8
aer 0.7192982456140351
s_a 0
p_a 1
aer 0.9545454545454546
s_a 0
p_a 1
aer 0.9821428571428571
s_a 0
p_a 1
aer 0.9285714285714286
s_a 0
p_a 0
aer 1.0
s_a 0
p_a 1
aer 0.9666666666666667
s_a 0
p_a 0
aer 1.0
s_a 0
p_a 0
aer 1.0
s_a 2
p_a 2
aer 0.0
s_a 0
p_a 1
aer 0.9230769230769231
s_a 3
p_a 4
aer 0.9146341463414634
s_a 3
p_a 4
aer 0.7878787878787878
s_a 0
p_a 1
aer 0.9473684210526316
s_a 0
p_a 0
aer 1.0
s_a 0
p_a 0
aer 1.0